In [3]:
%%capture
# update or install the necessary libraries
!pip install --upgrade google-search-results


In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from langchain.agents import Tool, load_tools
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from virtual_assistant.tools import BrowsingTool
from virtual_assistant.utilities import GoogleSerperAPIWrapper

In [3]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [4]:
# tool_names = ['google-serper']
# tools = load_tools(tool_names, llm=llm)

tools = [BrowsingTool(serper=GoogleSerperAPIWrapper())]

In [5]:
llm=ChatOpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent="chat-conversational-react-description", verbose=True, memory=memory)

In [6]:
agent_chain.run("what are some good dinners to make this week, if i like thai food?")



> Entering new AgentExecutor chain...
{
    "action": "Browsing internet tool",
    "action_input": "list of thai food dinners to make this week"
}
Observation: 5. Pad Thai 

Chances of you ordering Pad Thai in a Thai restaurant is pretty high, am I right? Why wait to go to the restaurant when you can whip up the same restaurant quality dish in under 30 minutes, right in your very own kitchen. 
Pad Thai is a medley of chicken, shrimp or tofu along with cilantro, peanuts, bean sprouts, scrambles eggs and rice noodles tossed with delicious pad Thai sauce. Easy weeknight meal when the Asian cravings hit hard. 
Get the Recipe @ tastesbetterfromscratch

6. Sticky Thai Chicken Wings 

If you think chicken wings are just an appetizer… think again. They are excellent when served as mains with a side of some Asian noodles or fried rice. These sticky chicken wings are glazed with delicious Thai flavored sticky sauce. It’s one of the easiest Thai Recipes that you can cook, with ingredients you 

'Here are some Thai food dinners you can make this week: Pad Thai, Sticky Thai Chicken Wings, Thai Chicken Buddha Bowls, 15-Minute Pad See Ew, Thai Pineapple Fried Rice, Tom Yum Soup, Thai Steak Salad, and Thai Red Curry Noodle Soup.'

In [7]:
agent_chain.run(input="tell me who won the world cup in 2022?")



> Entering new AgentExecutor chain...
{
    "action": "Browsing internet tool",
    "action_input": "Who won the world cup in 2022?"
}
Observation: 2022 World Cup / Champion :  Argentina national football team
Thought:{
    "action": "Final Answer",
    "action_input": "The winner of the 2022 World Cup is Argentina national football team."
}

> Finished chain.


'The winner of the 2022 World Cup is Argentina national football team.'

In [11]:
agent_chain.run('Oscar best movie in 2023.')



> Entering new AgentExecutor chain...
{
    "action": "Browsing internet tool",
    "action_input": "What was the Oscar best movie in 2023"
}
Observation: Academy Award for Best Picture / Winners (2023) :  Everything Everywhere All at Once
Thought:{
    "action": "Final Answer",
    "action_input": "The Academy Award for Best Picture winner in 2023 was 'Everything Everywhere All at Once'."
}

> Finished chain.


"The Academy Award for Best Picture winner in 2023 was 'Everything Everywhere All at Once'."